<a href="https://colab.research.google.com/github/xmesa002/DRRModules/blob/master/1_Introductory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_________

<img src="https://github.com/xmesa002/DRRModules/blob/master/imgs/DRRlogo.jpg?raw=1" width="350" />

# <b>Land Cover Classification with Google Earth Engine</b> 

## Introduction


_________

## Remote Sensing Data in Disaster Risk Models

Flood hazard models require spatially explicit runoff factors (Fig. 1).  Spatially explicit runoff values are determined from water evaporation and infiltration rates, which are a function of the land cover, land use and the soil characteristics of a location.  Hence, to produce a runoff factor (N) map it is essential to have a land cover map.

In this tutorial you will learn how to search, find and visualize remotely sensed imagery from different sources like Google Earth Engine and the USGS Global Visualization Viewer (GloVis).

<img src="./imgs/CAPRAflood.PNG">

The purpose of the following section of the course is to derive a land cover map from satellite imagery using differemt classification methods. The procedure for the classification process is outlined in Figure 2. 

The tutorial will use the open source software Google Earth Engine (GEE) and its Python Application Programming Interface (API). We will cover common tasks for data loading, image visualization and processing. The main steps of the work flow developed for this tutorial with relevant software indicated in () are:

1. Installation of Anaconda distributions including Spyder and Jupyter notebook plug-in<br>
2. Acquisition and Processing Options (Exploration: GloVis, USGS-EE; Download: USGS-EE, ESPA; Cloud Processing: GEE<br>
3. Image Evaluation and Visualization (GEE Python API)<br>
4. Classification Scheme Definition<br> 
5. Remote Sensing Process<br>
6. Derivation of Additional Information from Reflectance Image (GEE Python API)<br>
7. Evaluation of Variable Set and Classification Algorithm<br> - Digitization of Reference Locations for Training Set (Google Earth)<br> - Evaluation of Variable Set and Classification Algorithm (GEE Python API)<br> - Classification and evaluation of classifiers (GEE Python API)<br> - Generate Final Map (GEE Python API)<br>
8. Accuracy Assessment<br> - Stratified Random Sampling by Land-Cover Class (GEE Python API)<br> -  Visual Evaluation of Random Samples (Google Earth)<br> - Evaluation of the Contingency Matrix (GEE Python API)


<img src="./imgs/JensenClfProcess.PNG" width="700" />

## What is Google Earth Engine?

Gorelick et al. (2017) wrote in the 2017 <i>Remote Sensing of the Environment</i> article the following to describe GEE:

<i>Google Earth Engine is a cloud-based platform for planetary-scale geospatial analysis that brings Google’s massive computational capabilities to bear on a variety of high-impact societal issues including deforestation, drought, disaster, disease, food security, water management, climate monitoring and environmental protection.</i>

The purpose of Earth Engine is to perform highly-interactive algorithm development at a global scale; extend the limits for big data in remote sensing and enable high-impact, data driven science.

Through GEE you can run geospatial analysis  on Google's inrastrutucture. There are serveral ways to interact with the platform:

__[Explorer](https://explorer.earthengine.google.com/#workspace)__  
__[Code Editor](https://code.earthengine.google.com/)__<br>
__[Javascript library](https://github.com/google/earthengine-api/tree/master/javascript)__<br>
__[Python library]__

This course will focus on th elast option. The python wrapper library can be used to develop Earth Engine code locally. You can visit the Earth Engine's Developer Guide for an overview.

#### Components

The main components of Earth Engine are:

<b>Datasets:</b> A petabyte-scale archive of publicly available remotely sensed imagery and other data. Explore the data catalog.

<b>Compute power:</b> Google’s computational infrastructure optimized for parallel processing of geospatial data.

<b>APIs:</b> APIs for JavaScript and Python (hosted on GitHub) for making requests to the Earth Engine servers. These docs focus on JavaScript; (Get started guide for the Javascript API). For Python, see the Python install guide and the Python examples in the Earth Engine GitHub repository.

<b>Code Editor:</b> An online Integrated Development Environment (IDE) for rapid prototyping and visualization of complex spatial analyses using the Javascript API. Explore the Code Editor docs.

In [1]:
#@title ### Meet Earth Engine
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/gKGOeTFHnKY" frameborder="0" allowfullscreen></iframe>')


##  Connecting GEE with other Google Services


The Earth Engine Python API and command line tools can be installed using Python's pip package installation tool. The following notebook cell line starts with ! to indicate that a shell command should be invoked.

In [2]:
!pip install earthengine-api

### Authentication 

To read/write from a Google Drive account to which you have access, it's necessary to authenticate. You'll also need to authenticate with Earth Engine, so that you'll have access to your scripts, assets, etc.

#### Authenticate to Earth Engine

In order to access Earth Engine, signup at [signup.earthengine.google.com](https://signup.earthengine.google.com/).

When you run the code below, it will display a link in the output to an authentication page in your browser. Follow the link to a page that will let you grant permission to the Google Earth Engine to access your resources. Copy the code from the permissions page back into this notebook and press return to complete the process.



In [3]:
!earthengine authenticate

Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

/usr/local/lib/python3.6/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/wAFUMbXAiPiX_W93iV1ZZf65cqvduJXO1CYijxp6Z7S

#### Authenticate to Google Drive

Authenticate to **Google Drive** on  the same way you did to the Earth Engine.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


 Once you have authorized access (just Earth Engine auth is required). Initialize your connection as shown the cell below.

In [0]:
# Import the Earth Engine Python Package and initialize it.
import ee
ee.Initialize()